# PySpark Data Validation and Processing

This notebook demonstrates how to validate CSV data using PySpark, separating valid records from invalid ones. We'll work with a simple schema and handle corrupt records gracefully.

## Setup and Schema Definition

First, we'll import the necessary libraries and define our schema with a special `_corrupt_record` field that captures malformed rows.

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col

spark = SparkSession.Builder().appName("Handle Bad Data").getOrCreate()

# Define schema for validation (includes _corrupt_record field to capture malformed rows)
csv_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("_corrupt_record", StringType(), True)  # Captures rows that don't match schema
])

# Output locations
valid_output_path = "valid_records"
invalid_output_path = "invalid_records"

25/04/20 08:07:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Data Preparation

Next, we'll create RDDs with test data. We have two sources:
1. New data with some valid and invalid records
2. Previously invalid records that we're attempting to reprocess

In [11]:
# Create RDDs for test data
# In production, you would read from files instead of using test data
new_data_rdd = spark.sparkContext.parallelize([
    "1,John Doe,30",
    "2,Jane Smith,25",
    "3,Bob Johnson,35",
    "4,Alice Johnson,twenty"  # Invalid: age is not an integer
])

# Previously invalid records to reprocess
previous_invalid_rdd = spark.sparkContext.parallelize([
    "5,45,Charlie Brown",  # Invalid: id and age are position swapped
])

## Data Loading with Schema Validation

We'll load the data with our schema validation. The schema will automatically detect records that don't match the expected structure and place them in the `_corrupt_record` column.

In [12]:
# Read data with schema validation
new_df = spark.read.schema(csv_schema).csv(new_data_rdd)
previous_invalid_df = spark.read.schema(csv_schema).csv(previous_invalid_rdd)

# Combine datasets for unified processing
combined_df = new_df.union(previous_invalid_df)

# Show all data including corrupt records
print("All records:")
combined_df.show()

All records:


+---+-------------+----+--------------------+
| id|         name| age|     _corrupt_record|
+---+-------------+----+--------------------+
|  1|     John Doe|  30|                NULL|
|  2|   Jane Smith|  25|                NULL|
|  3|  Bob Johnson|  35|                NULL|
|  4|Alice Johnson|NULL|4,Alice Johnson,t...|
|  5|           45|NULL|  5,45,Charlie Brown|
+---+-------------+----+--------------------+



## Data Processing

Now we'll separate the valid records from the invalid ones using the `_corrupt_record` column.

In [13]:
# Separate valid from invalid records
valid_df = combined_df.filter(col("_corrupt_record").isNull()).drop("_corrupt_record")
invalid_df = combined_df.filter(col("_corrupt_record").isNotNull()).select("_corrupt_record")

## Writing Results

We'll write the valid records as CSV with headers, and the invalid records in their original form as text files.

In [14]:
# Write valid records as CSV with headers
valid_df.write.option("header", "true").mode("overwrite").csv(valid_output_path)

# Write invalid records in their original form as text
# Using text format because _corrupt_record contains the original CSV line as string
invalid_df.write.option("header", "false").mode("overwrite").text(invalid_output_path)

## Results Display

Finally, let's display our valid and invalid records to verify the processing.

In [15]:
# Show results
print("Valid records:")
valid_df.show()

print("Invalid records (original form):")
invalid_df.show()

Valid records:
+---+-----------+---+
| id|       name|age|
+---+-----------+---+
|  1|   John Doe| 30|
|  2| Jane Smith| 25|
|  3|Bob Johnson| 35|
+---+-----------+---+

Invalid records (original form):


+--------------------+
|     _corrupt_record|
+--------------------+
|4,Alice Johnson,t...|
|  5,45,Charlie Brown|
+--------------------+

